In [10]:
from transformers import pipeline
import pandas as pd
import torch
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [11]:
class TweetDataset(Dataset):
    def __init__(self, tweets, candidate_labels):
        self.tweets = tweets
        self.candidate_labels = candidate_labels
        self.classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
    
    def __len__(self):
        return len(self.tweets)
    
    def __getitem__(self, idx):
        tweet = self.tweets[idx]
        result = self.classifier(tweet, self.candidate_labels, multi_label=True, torch_dtype=torch.float16)
        return result

In [12]:
print("Reading data")
data = pd.read_csv("data/Russia_invade.csv")

Reading data


In [36]:
print("Creating dataloader")
tweets = data["renderedContent"].tolist()[:64]
candidate_labels = ['USA started the war',
                    'POTUS started the war',
                    'Joe Biden started the war',
                    'CIA started the war',
                    'USA influenced the war',
                    'POTUS influenced the war',
                    'Joe Biden influenced the war',
                    'CIA influenced the war',
                    'USA are the enemy',
                    'POTUS are the enemy',
                    'Joe Biden are the enemy',
                    'CIA are the enemy']
tweet_dataset = TweetDataset(tweets, candidate_labels)
tweet_dataloader = DataLoader(tweet_dataset, batch_size=4)


Creating dataloader


In [37]:

print("Starting analysis")
tweets_blaming_america = {}
for batch in tqdm(tweet_dataloader):
    results = {}
    for i, tweet in enumerate(batch['sequence']):
        labels = [l[i] for l in batch['labels']]
        scores = [s[i] for s in batch['scores']]
        results[tweet] = {label: score for label, score in zip(labels, scores)}
    
    for tweet, result in results.items():
        if any(val > 0.75 for val in result.values()):
            tweets_blaming_america[tweet] = result

Starting analysis


 56%|█████▋    | 9/16 [03:26<02:40, 22.94s/it]


KeyboardInterrupt: 

In [ ]:
for tweet, result in tweets_blaming_america.items():
    print(tweet.replace("\n", " "))
    for label, score in result.items():
        print(f"\t{label}: {score}")

America forced Putin to invade in order to bog down Russia. Because Americans are evil and are sitting in their White House cackling evilly, devising wicked plans.  10/10 genius logic Glenn https://t.co/zHHGG5amqX
	USA are evil: 0.9979836344718933
	USA started the war: 0.9240317940711975
	POTUS is bad: 0.7327097058296204
	Joe Biden is a bad President: 0.0019157660426571965
@NeilLowenthal1 Putin probably hoped that Trump got re-elected then withdrew from NATO, which would either dissolve or weaken the alliance and then open the door for Russia to invade not only Ukraine but also Latvia, Estonia, Georgia, Lithuania, Armenia, Azerbaijan, and Moldova (to name a few).
	POTUS is bad: 0.8312374949455261
	USA are evil: 0.1754041314125061
	USA started the war: 0.00015745432756375521
	Joe Biden is a bad President: 0.00014913138875272125
